**ИМПОРТ БИБЛИОТЕК И РЕСУРСОВ**

In [4]:
!pip install pandas nltk nlpaug scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.4 MB/s eta 0:00:00


In [6]:
import pandas as pd
import nltk
from nlpaug.augmenter.word import RandomWordAug
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

nltk.download('punkt')  # Токенизатор

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**ПРЕДОБРАБОТКА ТЕКСТА**

In [16]:
with open('spam.csv', encoding='latin-1') as f:
    lines = f.readlines()

labels = []
texts = []

for line in lines[1:]:  # пропускаем заголовок
    line = line.strip()
    if not line:
        continue
    # Разделяем по первой запятой, чтобы метка и текст были разделены
    parts = line.split(',', 1)
    if len(parts) == 2:
        label, text = parts
        labels.append(label.strip())
        texts.append(text.strip())
    else:
        # Если нет запятой — пропускаем
        continue

import pandas as pd
data = pd.DataFrame({'label': labels, 'text': texts})
data['label_num'] = data['label'].map({'ham': 0, 'spam': 1})

print(f"Количество обычных сообщений: {(data['label_num'] == 0).sum()}")
print(f"Количество спам-сообщений: {(data['label_num'] == 1).sum()}")

Количество обычных сообщений: 31
Количество спам-сообщений: 31


**АУГМЕНТАЦИЯ ТЕКСТА**

In [8]:
# Инициализируем аугментатор
augmenter = RandomWordAug(action="swap")

spam_texts = data.loc[data['label_num'] == 1, 'text'].values
augmented_texts = []

for text in spam_texts[:31]:  # аугментируем 31 сообщение
    try:
        augmented = augmenter.augment(text)
        augmented_texts.append(augmented)
    except Exception as e:
        # Пропускаем тексты, если аугментация не удалась
        continue

# Создаем полный список текстов и меток
all_texts = list(data['text'].values) + augmented_texts
all_labels = list(data['label_num'].values) + [1] * len(augmented_texts)

# Очищаем данные: удаляем пустые или нестроковые элементы
clean_texts = []
clean_labels = []

for text, label in zip(all_texts, all_labels):
    if isinstance(text, str) and text.strip() and pd.notna(label):
        clean_texts.append(text.strip())
        clean_labels.append(label)

**ВЕКТОРИЗАЦИЯ ТЕКСТА**

In [9]:
# Векторизация текста с поддержкой русского языка
vectorizer = TfidfVectorizer(lowercase=True, token_pattern=r'\b\w{2,}\b')
X = vectorizer.fit_transform(clean_texts)
y = clean_labels

# Разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучаем модель Наивного Байеса
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

**ОЦЕНКА КАЧЕСТВА МОДЕЛИ**

In [17]:
# Предсказания и отчет по классификации
predictions = model.predict(X_test)
print(classification_report(y_test, predictions, target_names=['ham', 'spam']))

              precision    recall  f1-score   support

         ham       0.67      1.00      0.80        10
        spam       0.00      0.00      0.00         5

    accuracy                           0.67        15
   macro avg       0.33      0.50      0.40        15
weighted avg       0.44      0.67      0.53        15



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
